In [ ]:
model_conf:
    input_dimension: 10
    output_dimension: 10 
    layers : [  ]



In [70]:
model_conf = {}
model_conf['input_dimension'] = 2
model_conf['output_dimension'] = 2 

model_conf['layers'] = [
    { "type":"linear", "dim_factor":2},
    { "type": "activation", "act_fun": "relu"},
    {  "type":"linear", "dim_factor":2},
    { "type": "activation", "act_fun": "relu"},
]
model_conf['layers'] = None 

In [71]:
import torch
from torch import nn 
from torch import optim
import torch.nn.functional as F  

In [74]:
class MLP(nn.Module):
    def __init__(self, model_conf):
        super(MLP, self).__init__()

        self.in_dim  = model_conf['input_dimension']
        self.out_dim = model_conf['output_dimension']
        #k       = model_conf['num_classes']
        layers = nn.ModuleList()

        if('layers' not in model_conf or model_conf['layers'] is None or len(model_conf['layers'])==0):
            layers.add_module('layer_0', nn.Linear(self.in_dim,self.out_dim))

        elif( 'layers' in model_conf and  len(model_conf['layers']) > 0):
            p_dim = [self.in_dim]

            for i in range(len(model_conf['layers'])):
                l_conf = model_conf['layers'][i]
                layer = None                 
                if(l_conf['type']=='linear'):
                    d2 = int(self.in_dim * l_conf['dim_factor'])
                    layer =  nn.Linear(  p_dim[i], d2)
                    p_dim.append(d2)

                elif(l_conf['type'] == 'activation'):
                    if(l_conf['act_fun']=='relu'):
                        layer =  nn.ReLU() 
                    elif(l_conf['act_fun']=='tanh'):
                        layer = nn.Tanh()
                    p_dim.append(p_dim[i])

                layers.add_module( f'layer_{i}' , layer)
            
            layers.add_module('last_layer', nn.Linear( p_dim[-1], self.out_dim) )


        self.net = nn.Sequential(*layers)

    def forward(self,x ):
        z = self.net(x)
        return z 

In [75]:
model = MLP(model_conf)
model 

MLP(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=2, bias=True)
  )
)

In [68]:
x = torch.tensor([0,0]).float() 

In [69]:
model.forward(x)

tensor([-0.1347,  0.2337], grad_fn=<AddBackward0>)

In [16]:
for param in model.parameters():
    print(param.data) # = nn.parameter.Parameter(torch.ones_like(param))
    param.data =  nn.parameter.Parameter(torch.ones_like(param))
    print(param.data)

tensor([[1., 1.],
        [1., 1.]])
tensor([[1., 1.],
        [1., 1.]])
tensor([1., 1.])
tensor([1., 1.])


In [26]:
torch.matmul( list(model.parameters())[0].data,x  )

tensor([0.3000, 0.3000])